# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [66]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv('../output_data/output.csv')
weather_data.drop('Unnamed: 0', axis=1, inplace=True)
weather_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,2112802,Hasaki,91,JP,1655102340,64,35.7333,140.8333,73.29,13.38
1,2075265,Busselton,85,AU,1655102479,69,-33.6500,115.3333,61.07,15.77
2,4021858,Guerrero Negro,29,MX,1655102479,92,27.9769,-114.0611,63.64,12.71
3,3652764,Puerto Ayora,86,EC,1655102480,91,-0.7393,-90.3518,71.55,1.01
4,2022572,Khatanga,100,RU,1655102480,27,71.9667,102.5000,72.30,5.59
...,...,...,...,...,...,...,...,...,...,...
609,2194098,Ahipara,31,NZ,1655102792,71,-35.1667,173.1667,56.86,22.68
610,935214,Saint-Pierre,75,RE,1655102793,88,-21.3393,55.4781,71.91,24.16
611,2206874,Waipawa,10,NZ,1655102793,79,-41.4122,175.5153,47.59,14.12
612,2155562,Nelson Bay,1,AU,1655102794,82,-32.7167,152.1500,60.01,1.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [86]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
marker_locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity, 
                                dissipating=False, max_intensity=0.5,
                                point_radius=6)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [78]:
nice_weather_data = weather_data.loc[(weather_data["Max Temp"] < 80) & (weather_data["Max Temp"] > 70)
                                    & (weather_data["Wind Speed"] < 10) & (weather_data["Cloudiness"] == 0)]

nice_weather_data["Hotel Name"] = ""

nice_weather_data

c:\Users\dokur\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
89,4862034,Iowa City,0,US,1655102525,83,41.6611,-91.5302,75.15,8.05,
174,2017155,Saskylakh,0,RU,1655102569,24,71.9167,114.0833,71.55,7.76,
294,2142316,Yeppoon,0,AU,1655102634,56,-23.1333,150.7333,71.38,4.21,
297,145724,Lucapa,0,AO,1655102635,38,-8.4192,20.7447,71.74,7.38,
321,2065594,Mount Isa,0,AU,1655102648,17,-20.7333,139.5000,73.17,6.91,
331,3128382,Béjar,0,ES,1655102654,32,40.3864,-5.7634,72.57,3.18,
332,3128382,Béjar,0,ES,1655102654,32,40.3864,-5.7634,72.57,3.18,
358,3347019,Mossamedes,0,AO,1655102668,54,-15.1961,12.1522,70.03,8.61,
381,3124765,Colmenar Viejo,0,ES,1655102680,36,40.6591,-3.7676,75.94,1.14,
390,3186952,Zadar,0,HR,1655102685,41,44.1197,15.2422,79.41,2.30,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [81]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in nice_weather_data.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    try:
        nice_weather_data.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("hotel not found")

c:\Users\dokur\anaconda3\envs\bootcamp\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


hotel not found
hotel not found


In [84]:
hotel_df = nice_weather_data

In [94]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [100]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info, 
                                display_info_box=True)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))